In [1]:
#Loading Packages
import numpy as np
import re
import string
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import openpyxl
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#Creating path for data extraction
now = datetime.now()
extraction_date = now.date()
date = extraction_date.strftime("%Y%m%d")
path = 'C:/Users/JenarthananRajenthir/OneDrive - ADA Asia/Documents/Final_telco/Data/MY/Hotlink/'              
if not os.path.exists(path):
    os.makedirs(path)
    
Carrier = 'Hotlink'

In [3]:
################### Explore Mobile #####################################
df_Mobile = pd.DataFrame(columns=['Carrier','Package_Type', 'Package_Option', 'Package_Name','Data', 'Calls',
                                   'SMS', 'Benefits', 'Info', 'Validity', 'Price'])

Package_Type = 'Mobile'

In [4]:
url1 = 'https://store.hotlink.com.my/#prepaidPlans'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
options.add_argument('--disable-gpu')
options.add_argument("--incognito")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument("--disable-blink-features=AutomationControlled")
browser = webdriver.Chrome(executable_path='../../chromedriver.exe', options = options)
browser.get(url1)
browser.implicitly_wait(10)
html_source_code = browser.execute_script("return document.innerHTML;")
browser.set_page_load_timeout(10)

html = browser.page_source
soup1 = BeautifulSoup(html, "html5lib")
browser.implicitly_wait(10)

Package_Option = 'Prepaid'
for i in soup1.find_all('div', {'class' : 'card-block'}):
    Package_Name = i.find_all('p')[1].text
    Price = i.find_all('span')[0].text
    Data = None
    Validity = None
    
    if len(i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})) == 1:
        Info = i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})[0].text
        Calls = None
        SMS = None
        Benefits = None
        
    elif len(i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})) == 2:
        Info = None
        Calls = i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})[0].text
        SMS = i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})[1].text
        Benefits = None
        
    elif len(i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})) == 3:
        Info = None
        Calls = i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})[0].text
        SMS = i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})[1].text
        Benefits = i.find_all('div', {'class' : 'fs-12 maxis-bold mb-2 lh-100 text-brown-grey'})[2].text
    
    else:
        continue 
    
    df_Mobile = df_Mobile.append({'Carrier': Carrier,'Package_Type' : Package_Type,'Package_Option' : Package_Option,
                                      'Package_Name' : Package_Name,'Data' : Data, 'Calls' : Calls, 'SMS' : SMS, 
                                      'Benefits' : Benefits, 'Validity' : Validity, 'Price' : Price, 'Info' : Info },
                                     ignore_index = True)
    
df_Mobile.head(7) 

<ipython-input-4-ad0a11fe570c>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='../../chromedriver.exe', options = options)


,Carrier,Package_Type,Package_Option,Package_Name,Data,Calls,SMS,Benefits,Info,Validity,Price
0,Hotlink,Mobile,Prepaid,Prepaid Unlimited,None,RM5 pre-loaded credit (valid for 5 days),FREE 500MB Basic Internet,None,None,None,RM5 pre-loaded credit (valid for 5 days)
1,Hotlink,Mobile,Prepaid,Prepaid Fast,None,None,None,None,RM5 pre-loaded credit (valid for 5 days),None,RM5 pre-loaded credit (valid for 5 days)
2,Hotlink,Mobile,Prepaid,Prepaid Fast,None,Call: 30 sen / 60 secs,SMS: 15 sen / SMS,None,None,None,Call: 30 sen / 60 secs
3,Hotlink,Mobile,Prepaid,Prepaid Unlimited,None,Call: 15 sen / 30 secs,SMS: 15 sen / SMS,500MB FREE Basic Internet,None,None,Call: 15 sen / 30 secs


In [5]:
url2 = 'https://www.hotlink.com.my/en/plan/switch-to-hotlink/'

html = requests.get(url2)
soup2 = BeautifulSoup(html.content, "html5lib")

Package_Option = 'Postpaid'
for i in range(0, 3):
    Package_Name = soup2.find_all('div', {'class' : 'hotlink-plans-card'})[i].find_all('p')[2].text
    Data = soup2.find_all('div', {'class' : 'hotlink-plans-card'})[i].find_all('p')[3].text
    Price = soup2.find_all('div', {'class' : 'hotlink-plans-card'})[i].find_all('p')[5].text
    Validity = None
    
    DS = soup2.find_all('div', {'class' : 'hotlink-plans-card'})[i].find_all('div', {'class' : 'description'})[0].text.split('\n')
    y =  [x.strip() for x in DS if x.strip()]
    
    if len(y) == 5:
        del y[2]
        
    Benefits = y[1]
    Info =  y[0]
    Calls = y[2]
    SMS = y[2]
    


    df_Mobile = df_Mobile.append({'Carrier': Carrier,'Package_Type' : Package_Type,'Package_Option' : Package_Option,
                                  'Package_Name' : Package_Name,'Data' : Data, 'Calls' : Calls, 'SMS' : SMS, 
                                  'Benefits' : Benefits, 'Validity' : Validity, 'Price' : Price, 'Info' : Info },
                                 ignore_index = True)
    
    
df_Mobile.head(10) 

,Carrier,Package_Type,Package_Option,Package_Name,Data,Calls,SMS,Benefits,Info,Validity,Price
0,Hotlink,Mobile,Prepaid,Prepaid Unlimited,None,RM5 pre-loaded credit (valid for 5 days),FREE 500MB Basic Internet,None,None,None,RM5 pre-loaded credit (valid for 5 days)
1,Hotlink,Mobile,Prepaid,Prepaid Fast,None,None,None,None,RM5 pre-loaded credit (valid for 5 days),None,RM5 pre-loaded credit (valid for 5 days)
2,Hotlink,Mobile,Prepaid,Prepaid Fast,None,Call: 30 sen / 60 secs,SMS: 15 sen / SMS,None,None,None,Call: 30 sen / 60 secs
3,Hotlink,Mobile,Prepaid,Prepaid Unlimited,None,Call: 15 sen / 30 secs,SMS: 15 sen / SMS,500MB FREE Basic Internet,None,None,Call: 15 sen / 30 secs
4,Hotlink,Mobile,Postpaid,Postpaid 30,2GB,Zero Advance Payment,Zero Advance Payment,Unlimited Calls & SMS,High-speed internet,None,RM30/mth
5,Hotlink,Mobile,Postpaid,Postpaid 40,,Zero rate plan advanced payment,Zero rate plan advanced payment,High-speed Internet,12GB 20GB,None,High-speed Internet\nUnlimited Calls & SMS\nZe...
6,Hotlink,Mobile,Postpaid,Postpaid 60,,Zero rate plan advanced payment,Zero rate plan advanced payment,High-speed Internet,30GB 40GB,None,High-speed Internet\nUnlimited Calls & SMS\nZe...


In [6]:
################### Explore Broadband #####################################
df_Broadband = pd.DataFrame(columns=['Carrier','Package_Type', 'Package_Option', 'Package_Name','Data', 'Calls',
                                   'SMS', 'Benefits', 'Info', 'Validity', 'Price'])

Package_Type = 'Broadband'

In [7]:
url3 = 'https://www.hotlink.com.my/en/products/postpaid-broadband/'

html = requests.get(url3)
soup3 = BeautifulSoup(html.content, "html5lib")

Package_Option = 'Postpaid Broadband'
for k in range(len(soup3.find_all('div', {'class' : 'hotlink-plans-card'}))):
    i = soup3.find_all('div', {'class' : 'hotlink-plans-card'})[k]
    Name = i.find_all('p')[1].text
   
    if Name == '':
        Package_Name = i.find_all('p')[2].text
        
    elif k == 6:
        Package_Name = i.find_all('p')[2].text+' '+i.find_all('p')[3].text+' '+i.find_all('p')[4].text
    
    elif k in range(6, 11):
        Package_Name = i.find_all('p')[1].text+' + '+i.find_all('p')[2].text+' '+i.find_all('p')[3].text+' '+\
        i.find_all('p')[4].text
        
    else:
        Package_Name = Name
        
    xlast = len(i.find_all('p')) - 1   # Since last 'p' hold the price value
    Price = i.find_all('p')[xlast].text
    Validity = None
    Calls = None
    SMS = None
    
    
    if xlast > 8:
        Data = i.find_all('p')[5].text  
    else:
        Data = None
        
    if len(i.find_all('div', {'class': 'benefits'})) == 3:
        Benefits = [x.strip() for x in i.find_all('div', {'class': 'benefits'})[0].text.split('\n         ') if x.strip()][0]+' \ '+\
        [x.strip() for x in i.find_all('div', {'class': 'benefits'})[2].text.split('\n         ') if x.strip()][0]
        Info =  [x.strip() for x in i.find_all('div', {'class': 'benefits'})[1].text.split('\n         ') if x.strip()][0]
        
    elif len(i.find_all('div', {'class': 'benefits'})) == 2:
        Benefits = [x.strip() for x in i.find_all('div', {'class': 'benefits'})[0].text.split('\n         ') if x.strip()][0]
        Info =  [x.strip() for x in i.find_all('div', {'class': 'benefits'})[1].text.split('\n         ') if x.strip()][0]
        
    else:
        continue
     
    
    df_Broadband = df_Broadband.append({'Carrier': Carrier,'Package_Type' : Package_Type,'Package_Option' : Package_Option,
                                        'Package_Name' : Package_Name,'Data' : Data, 'Calls' : Calls, 'SMS' : SMS, 
                                        'Benefits' : Benefits, 'Validity' : Validity, 'Price' : Price, 'Info' : Info },
                                       ignore_index = True)
    
    
df_Broadband.head(17)

,Carrier,Package_Type,Package_Option,Package_Name,Data,Calls,SMS,Benefits,Info,Validity,Price
0,Hotlink,Broadband,Postpaid Broadband,Maxis Fibre 30Mbps,None,None,None,FREE Next-gen WiFi 6 certified router,New Home Devices from RM29/mth,None,RM89/mth
1,Hotlink,Broadband,Postpaid Broadband,Maxis Fibre 100Mbps,None,None,None,FREE Next-gen WiFi 6 certified router,New Home Devices from RM19/mth,None,RM129/mth
2,Hotlink,Broadband,Postpaid Broadband,Maxis Fibre 300Mbps,None,None,None,FREE Next-gen WiFi 6 certified router,New Home Devices from RM9/mth,None,RM149/mth
3,Hotlink,Broadband,Postpaid Broadband,Maxis Fibre 500Mbps,None,None,None,FREE Next-gen WiFi 6 certified router \ FREE 2...,New Home Devices from RM1/mth,None,RM219/mth
4,Hotlink,Broadband,Postpaid Broadband,Maxis Fibre 800Mbps,None,None,None,FREE Next-gen WiFi 6 certified router \ FREE 2...,New Home Devices from RM1/mth,None,RM299/mth
5,Hotlink,Broadband,Postpaid Broadband,Maxis Home 4G WiFi,None,None,None,Unlimited Home 4G WiFi data,New Home Devices from RM29/mth,None,RM138/mth
6,Hotlink,Broadband,Postpaid Broadband,Postpaid 60 Maxis Fibre 30Mbps,,None,None,FREE Next-gen WiFi 6 certified router,New Home Devices from RM29/mth,None,RM149/mth
7,Hotlink,Broadband,Postpaid Broadband,Postpaid 60 + Maxis Fibre 100Mbps,40GB full speed Internet!,None,None,FREE Next-gen WiFi 6 certified router,New Home Devices from RM19/mth,None,RM189/mth
8,Hotlink,Broadband,Postpaid Broadband,Postpaid 60 + Maxis Fibre 300Mbps,40GB full speed Internet!,None,None,FREE Next-gen WiFi 6 certified router,New Home Devices from RM9/mth,None,RM209/mth
9,Hotlink,Broadband,Postpaid Broadband,Postpaid 60 + Maxis Fibre 500Mbps,40GB full speed Internet!,None,None,FREE Next-gen WiFi 6 certified router \ FREE 2...,New Home Devices from RM1/mth,None,RM279/mth


In [8]:
##### Extracing all data to an excel sheet #####
path ='C:/Users/JenarthananRajenthir/OneDrive - ADA Asia/Documents/Final_telco/Data/MY/Hotlink/' +date + '/'
df_Broadband.to_csv(path+date+'Broadband.csv', index=False)
df_Mobile.to_csv(path+date+'Mobile.csv', index=False)